# Making sure your data is BIDS-compliant on Flywheel

BIDS compliance is absolutely necessary for running many pre-processing pipelines on your data. Having correct metadata ensures that the pre-processing correctly handles your data. Incorrect BIDS naming can result in difficult-to-debug errors.

For these reasons, we cover in-depth how to make sure your data is BIDS-compliant.

First, let's look at an example subject. 

<img src="images/initial_bids_error.png" alt="drawing" width="600"/>

These red flags mean that there are errors converting the images to a BIDS-compliant name. Here you have two options

  1. Click the "Info" button on the scan and make sure the BIDS section is completed correctly
  2. Use the Flywheel API to change the BIDS properties


## The Flywheel data model

Flywheel stores zip archives of the dicom data and nifti files. The names of the nifti files are usually determined by `dcm2niix` and that name sticks with the file for the rest of its life. I believe these files are managed using MongoDB's GridFS framework, so there are some non-intuitive things that can happen. 

  - More than one file can exist with the exact same name. The files are indexed by a BSON id, which is
    unique for each file. The field containing the file name is not necessarily unique.

  - Unless the data was uploaded in BIDS format, the actual BIDS-named data does not exist in Flywheel.
    Instead, when a job needs requires BIDS input, the job uses the Flywheel API to download and rename
    the data to be BIDS compliant.

    - The renaming and JSON Sidecar creation uses information stored in Flywheel's MongoDB. Therefore to
      change the BIDS filename, you simply update the files Document/BSON and next time a gear is run,
      this new metadata will be used to create the BIDS directory.

The metadata associated with each file can be edited through the Flywheel web interface or through the API. The web interface is not complete and still a little buggy (i.e. not all UI's contain BIDS fields). The SDK is fairly complete and documented, so that will be the focus of this notebook.
        

## Editing BIDS info from the web interface

You can change the information on your scans by clicking the info icon in the file's row.

<img src="images/select_info.png" alt="drawing" width="600"/>


The fields can then be edited manually. This is the initial interface presented:

<img src="images/basic_file_info.png" alt="drawing" width="600"/>

where basic options about the contents of the file are editable. This will always be available when clicking the Info button. If the `BIDS Curation` gear has been run, you can scroll down and see the BIDS fields:

<img src="images/bids_info.png" alt="drawing" width="600"/>

It is unclear to me if the `BIDS Curation` gear needs to be re-run for these changes to be committed to MongoDB.

## Editing BIDS info from the API

The Flywheel SDK can be installed from PyPI using

```bash

pip install flywheel-sdk

```

The documentation for this package can be found [here](https://flywheel-io.github.io/core/branches/master/python/index.html). 

In [1]:
import flywheel

# Create client
fw = flywheel.Client()

## Logging In to Flywheel

Note the documentation is out of date and the constructor for `flywheel.Client` does not take any arguments. Instead you have to login with your API key using the commandline `fw login`. Verify that your login worked and you can retrieve data from Flywheel:

In [2]:
fw.get_current_user()['email']

'tinashemtapera@gmail.com'

## The Basic Data Model

Everything in flywheel is a *container*. >>>

<img src="images/data-model.png" alt="drawing" width="600"/>

When you access a container, you can either do so as a method attribute, or a function. With a method, you implement a flywheel finder which indexes the data but doesn't quite return anything to you yet. As a function, the returned object usually is a dictionary (or list of dictionaries).

For example, we can see the currently available project containers with the following command:

In [59]:
# the function
fw.projects()

[{'analyses': None,
  'created': datetime.datetime(2018, 12, 13, 18, 23, 41, 986000, tzinfo=tzutc()),
  'description': None,
  'files': [],
  'group': 'mcieslak',
  'id': '5c12a3ad9011bd0013368726',
  'info': {},
  'info_exists': False,
  'label': 'BBLTest',
  'modified': datetime.datetime(2019, 2, 7, 17, 39, 35, 998000, tzinfo=tzutc()),
  'notes': [],
  'parents': {'acquisition': None,
              'analysis': None,
              'group': 'mcieslak',
              'project': None,
              'session': None,
              'subject': None},
  'permissions': [{'access': 'admin', 'id': 'tinashemtapera@gmail.com'}],
  'public': None,
  'tags': [],
  'template': None}, {'analyses': None,
  'created': datetime.datetime(2018, 12, 19, 15, 21, 10, 217000, tzinfo=tzutc()),
  'description': None,
  'files': [{'classification': {},
             'created': datetime.datetime(2019, 1, 9, 19, 48, 52, 13000, tzinfo=tzutc()),
             'hash': 'v0-sha384-1c77469675dad45012de0cbf1da5053e27918713a

In [58]:
# the indexer
fw.projects

## Searching the Data

The indexer itself has some common search methods such as `find()` and `find_first()`.

In [25]:
project = fw.projects.find_first('label=Reward2018')
print(project)

{'analyses': None,
 'created': datetime.datetime(2018, 12, 19, 15, 21, 10, 217000, tzinfo=tzutc()),
 'description': None,
 'files': [{'classification': {},
            'created': datetime.datetime(2019, 1, 9, 19, 48, 52, 13000, tzinfo=tzutc()),
            'hash': 'v0-sha384-1c77469675dad45012de0cbf1da5053e27918713ad822de64502a711d0238e56bc7b3f94946ba1e8d1921d1c8d68d1d4',
            'id': '66463c4e-c46b-4975-868d-fb48c45039e8',
            'info': {u'BIDS': {u'Filename': u'',
                               u'Folder': u'',
                               u'Path': u'',
                               u'error_message': u"Filename u'' is too short",
                               u'ignore': False,
                               u'template': u'project_file',
                               u'valid': False}},
            'info_exists': True,
            'mimetype': 'text/plain',
            'modality': None,
            'modified': datetime.datetime(2019, 1, 9, 22, 38, 19, 579000, tzinfo=tzutc

Once you can find a container by label, use the BSON id to extract its other key-value pairs. You'll notice that there is an `'_id'` field in all of these documents. This is the BSON id that is used to identify files in flywheel. Everything - files, subjects, sessions, analyses - has one of these that can be used for direct access. Confusingly, you can't access it directly with `project['id']`: you'll need to use `project['_id']`.

In [26]:
project['_id']

'5c1a61e69011bd0011368884'

In MongoDB, containers don't have to be accessed in order. For example, we can pull the subjects out of the project here:

In [29]:
for subject in project.subjects():
        print('%s: %s' % (subject.id, subject.label))

5c352b991de80b0024480dda: 10180
5c352a081de80b0024480dcc: 102102
5c3535641de80b0024480e41: 104059
5c3527fe1de80b0024480db7: 10410
5c3541301de80b0024480ebc: 105168
5c353ab01de80b00198acafb: 105272
5c3542de1de80b00198acb34: 105490
5c35186f1de80b00198ac9f9: 105634
5c3539ea1de80b001c0da9c8: 106573
5c353e151de80b0024480ea0: 107055
5c3545391de80b00198acb42: 109741
5c3511cd1de80b0024480c9f: 11010
5c3532511de80b0024480e26: 11176
5c351c2f1de80b00156d3d76: 11186
5c1a8b619011bd0011369953: 11242
5c1a8b249011bd001436aa02: 11305
5c1a8acd9011bd0015369b48: 113220
5c1a89509011bd001436a68d: 11399
5c1a88959011bd0013369c49: 11419
5c1a876b9011bd0015369675: 11569
5c1a87089011bd0013369978: 11588
5c1a85f79011bd0014369b3b: 11599
5c1a8b2d9011bd0015369ba6: 11647
5c1a8a939011bd0013369f79: 116531
5c1a8a2c9011bd0011369848: 11706
5c1a89b69011bd001436a741: 117256
5c1a88cf9011bd001436a556: 11762
5c1a87df9011bd001436a2f3: 11801
5c1a86b89011bd0015369557: 11866
5c1a863c9011bd00113693cc: 118990
5c1a85ae9011bd0014369914: 1

And the sessions from a project as shown below. Sessions are available as child nodes of the project container. You can access a child node by using the `.` attribute on a container. Here, we use `find_first()`.

In [89]:
sessions = project.sessions.find_first()
sessions

{'age': None,
 'analyses': None,
 'created': datetime.datetime(2018, 12, 19, 17, 21, 42, 780000, tzinfo=tzutc()),
 'files': [],
 'group': 'mcieslak',
 'id': '5c1a7e269011bd0011368995',
 'info': {u'BIDS': {u'Label': u'day2',
                    u'Subject': u'17378',
                    u'ignore': False,
                    u'template': u'session'}},
 'info_exists': True,
 'label': 'day2',
 'modified': datetime.datetime(2018, 12, 19, 17, 55, 51, 723000, tzinfo=tzutc()),
 'notes': [],
 'operator': 'TK',
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'mcieslak',
             'project': '5c1a61e69011bd0011368884',
             'session': None,
             'subject': '5c1a7e269011bd0011368994'},
 'permissions': [{'access': 'admin', 'id': 'tinashemtapera@gmail.com'}],
 'project': '5c1a61e69011bd0011368884',
 'project_has_template': None,
 'public': None,
 'satisfies_template': None,
 'subject': {'age': None,
             'analyses': None,
             

Notice that the use of `()` on a container returns an iteratable list, but without the function parentheses, the container is an in-built attribute that indexes the flywheel database:

In [30]:
type(project.subjects())

list

In [31]:
type(project.subjects)

flywheel.finder.Finder

The `get()` function can be used to go directly to a container:

In [32]:
subject = fw.get("5c1a82ca9011bd0015368ee6")

And while containers are not quite Russian nesting dolls, they can still be treated as such (project > subject > session > aqsuisition)

In [33]:
for ses in subject.sessions():
    print('%s: %s' % (ses.id, ses.label))

5c1a82ca9011bd0015368ee7: neff2


In [34]:
ses1 = fw.get("5c1a82ca9011bd0015368ee7")

In [35]:
ses1.acquisitions()

[{'analyses': None,
  'collections': None,
  'created': datetime.datetime(2018, 12, 19, 17, 41, 30, 628000, tzinfo=tzutc()),
  'files': [{'classification': {},
             'created': datetime.datetime(2018, 12, 19, 17, 43, 38, 919000, tzinfo=tzutc()),
             'hash': '',
             'id': '5c728e0c-3f7e-4acf-84f2-8aed91daf589',
             'info': {u'BIDS': u'NA'},
             'info_exists': True,
             'mimetype': 'application/zip',
             'modality': 'MR',
             'modified': datetime.datetime(2018, 12, 19, 21, 57, 49, 252000, tzinfo=tzutc()),
             'name': 'B0map_onesizefitsall_v4.dicom.zip',
             'origin': {'id': 'mattcieslak@gmail.com',
                        'method': None,
                        'name': None,
                        'type': 'user',
                        'via': None},
             'replaced': None,
             'size': 1657355,
             'tags': [],
             'type': 'dicom',
             'zip_member_count': Non

File are accessed via the `files` key, but it's easier to use `get()` on the BSON id.

In [49]:
f1 = fw.get("5c1a82ca9011bd0011368ed1")

In [50]:
f1.label

'foo'

## Viewing the data as Tabular

Using `View()`, you can create a tabular index of the data you want to search through. From there, use `pandas` to read it in as a dataframe:

In [68]:
import pandas as pd

view = fw.View(columns='subject')
df = fw.read_view_dataframe(view,project.id)
df

,project.id,project.label,subject.cohort,subject.ethnicity,subject.firstname,subject.id,subject.label,subject.lastname,subject.race,subject.sex,subject.species,subject.strain,subject.type
0,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a70239011bd001436894c,100088,NaN,NaN,None,None,None,None
1,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd0014368976,90683,NaN,NaN,female,None,None,None
2,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd001436897a,91460,NaN,NaN,male,None,None,None
3,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd00153688a4,93204,NaN,NaN,female,None,None,None
4,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd00153688a6,93274,NaN,NaN,female,None,None,None
5,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd001436897c,94028,NaN,NaN,male,None,None,None
6,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd0014368978,98718,NaN,NaN,male,None,None,None
7,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c1a754e9011bd00113688be,99949,NaN,NaN,female,None,None,None
8,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c2655264a9bad0001de981f,90683-deleted,NaN,NaN,None,None,None,None
9,5c1a61e69011bd0011368884,Reward2018,NaN,NaN,NaN,5c2655264a9bad0001de982b,98718-deleted,NaN,NaN,None,None,None,None


In this implementation, there is a clear hierarchy to be followed. Basically, you specify the columns of the minor container you want to return, as members of some major container you have already assigned in the environment. So for example, if I wanted to see the name and size of all of the files of a session from a participant:

In [99]:
f1.id

'5c1a82ca9011bd0011368ed1'

In [ ]:

view = fw.View(columns='file')
df = fw.read_view_dataframe(view,f1.id)
df

In [94]:
fw.print_view_columns()

project (group): All column aliases belonging to project
project.id (string): The project id
project.label (string): The project label
project.info (object): The freeform project metadata
subject (group): All column aliases belonging to subject
subject.id (string): The subject id
subject.label (string): The subject label or code
subject.firstname (string): The subject first name
subject.lastname (string): The subject last name
subject.sex (string): The subject sex (one of female|male|other|unknown)
subject.race (string): The subject race (one of American Indian or Alaska Native|Asian|Native Hawaiian or Other Pacific Islander|Black or African American|White|More Than One Race|Unknown or Not Reported)
subject.ethnicity (string): The subject ethnicity (one of Not Hispanic or Latino|Hispanic or Latino|Unknown or Not Reported)
subject.cohort (string): The subject cohort (one of Control|Study|Training|Test|Validation)
subject.type (string): The subject type (one of human|animal|phantom)
subj

In [91]:
view = fw.View(columns='session')
df = fw.read_view_dataframe(view, ses.id)
df

ApiException: (500) Reason: Internal Server Error

## Manipulating Data Values

We can manipulate field values by passing a key-value pair to the `update()` function. Then, to update the object in the python environment, simply reassign it with `reload()`:

In [55]:
newname = {"label":"foo1"}
f1.update(newname)
f1 = f1.reload()

In [56]:
f1.label

'foo1'